# Kalman Filters 

Kalman filters are an important modelling topic in computer vision and robotics. We use it when we need to obtain dynamic measurements from sensors and predict or localize the world around us. 

Like a robot, we use sensors to percieve the world. When a bike zooms by on the same sidewalk your on, your sight and hearing do some pretty amazing sensor fusion to localize, track, and predict the location of that bike, so that you can steer clear into safety. A robot is similar. 

Robots use radar, lidar, maybe cameras or microphones etc. to percieve the world. We know that these sensors are noisy, because, well, we built them. The wheel encoder on your new robot says it drove 3 meters forward - take a ruler out and low and behold, it actually went 3 +/- 0.2 meters forward! (depending how accurate your ruler is). Enter the Kalman Filter. 

The Kalman Filter is a mathematical tool that uses probability theory, and a bit of linear algebra in some cases, to combine sensors readings to build a more accurate model of the world than any one sensor would give. It basically boils down to this: 

<img src="img/venn.png" width="400" border="10" />

If person 1 believes something with X probability, and person 2 believes something with Y probablity, then their shared belief has a probablity greater than either X or Y. If these are sensor measurements about a passing bike, then the fused probablities from the sensors have a stronger idea of where the bike is than either alone, even if both sensors are bad. 

This is pretty amazing if you let it soak in. We can take two noisy measurements, and rather non-intuitively, combining them builds certainty. How does this work? Gaussians. Beautiful, magical gaussians. 

### Gaussians 

<img src="img/gaussian.png" width="300" border="10" />

Beautiful, is it not? Here’s the equation for that little bump: 

<img src="img/gaussian_formula.png" width="300" border="10" />

The gaussian curve is defined by its mean, $\mu$, and its standard deviation, $\sigma$. The Gaussian is a probability curve, the area under the curve adds up to one, and the peak is centered on the mean. A curve with a low standard deviation, a measure of variance, will be taller, and more certain. A curve with high variance will be flat and uncertain. Mathematically, gaussians truly shine. The product of two gaussians? A Gaussian. The sum, convolution or fourier transform? A gaussian! 

The *truly* great part however, is that real world events can often be modeled by Gaussians, such as the average height of people around the world, or the noise in a radar sensor. Here is a toy exmaple a made up in Python. As you can see, the injected noise into the sensor can be roughly modeled by this Gaussian. Mathematically, this is a nice assumption to make.  

<img src="img/radar_measurements.png" width="800" border="10" />

#### Kalman Filter 1-D case 

So now imagine we had two noisy sensors like this. Each one has a slightly different $\mu$, or measurement of how far it says it is from the wall, and $\sigma$, or error in its measurement. We'll call reading in these data points the **measurment step**.

Now, we need to combine these measurements to get a better estimate of the true location. To do that, we'll [multiply the two gaussians](https://math.stackexchange.com/questions/157172/product-of-two-multivariate-gaussians-distributions), which we can simplify to the manipulation of just the $\mu$ and $\sigma$: 

$ \sigma_3  =  \frac{1}{(\sigma_1^{-1} + \sigma_2^{-1})}$ and 
$ \mu_3 = \frac{\sigma_1^{2}\mu_2 + \sigma_2^{2}\mu_1} {\sigma_1^{2} + \sigma_2^{2}}$

<img src="img/update.png" width="800" border="10" />

This is where things get interesting. We have our two noisy measurements, the blue and orange curves. Now for any given time, we can update our measurements using the two formulas above to generate a new estimate who's mean is an average of the two, but who's certainty is higher! Look how high the probablity is for the green curve. 

Another important thing to note is that this filtering requires very little data. Just our past prediction and the past measurements is used on a single iteration. A moving average on the other hand requires substantinally more data. 

So in review, this is the essence of the Kalman Filter. 
```python 
while True:
```
    1. Make a prediction of a state, based on some previous values and model.
    2. Obtain the measurement of that state, from sensor.
    3. Update our prediction, based on our errors
    4. Repeat.

### Extending to Higher Dimensions 

There are multiple directions to go from here. Since I am writing this to accompany my project with the Udacity Self-Driving-Car course, I will discuss prediction for higher dimensions, and with non-linear data (and so the use of the Extended Kalman Filter). To do that, we will need to utilize Linear Algebra to generalize the math. 

A self-driving car typically uses a combination of lidar and radar to model the world. A lidar can measure distance of a nearby objects that can easily be converted to cartesian coordinates (px, py) . A radar sensor can measure speed within its line of sight (drho) using the doppler effect. It can also measure distance of nearby objects that can easily be converted to polar coordinates (rho, phi) but in a lower resolution than lidar. 

For now we will work the position (p) and velocity (v) data. Note a = acceleration, t = time, and ' = update. 

$$\begin{array}{lcr}
    p_x^{'} = p_x + v_x \Delta t + \frac{a_x \Delta t^{2}}{2}\\
    p_y^{'} = p_y + v_y \Delta t + \frac{a_y \Delta t^{2}}{2}\\
    v_x^{'} = v_x + a_x \Delta t \\
    v_y^{'} = v_y + a_y \Delta t \\
\end{array}$$

Accelerations are assumed to be constant, so anything resdual will actually be processed as noise. Later on this will be adddressed with the Unscented Kalman Filter. 

For now we can throw these equations into the `F` matrix, which describes the estiamte of the current state of the object.  
    
$F = \begin{bmatrix}
    1 & 0 & \Delta t & 0  \\
    0 & 1 & 0 & \Delta t  \\
    0 & 0 & 1 & 0  \\
    0 & 0 & 0 & 1  \\
\end{bmatrix}$ 

Additioanlly, we use this list of vecotrs and matrices:

- x, state estimate matrix
- P process, or covariance matrix
    - describes the uncertainty of `x`, same dimensions as `x`.
- F state transition matrix
    - used to predict the next `x` and `P`, (4x4). 
- Q is the process noise covariance matrix
    - accounts for added noise for things like acceleration 
    - Q noise increases as $\Delta t$ between measurements increase

### Linear Algebra aside 

Let's take a second to talk about what the diagonals in matrices mean. They are covariances: how much to do two elements of a matrix vary with eachother? If I observe that as the magnitude of position `py` grows, the velocity `vx`grows as well, then `py` and `vx` are correalted. No pattern means zero correlation.

The diagonal of P contains the variance of that element with itself, which is always one. Around the diagonals are the covariances between two state elements, which is how the values of the state elements correlate with each other. All covariance matrices such as P, R, S, and Q are symmetric matrices, because the covariance of py and vx is also the covariance of vx and py.

### Back to Kalman Filters 
    
### Prediction 

With these matrices we can make a prediction of the state estimate and uncertainty (a priori to the measurements): 

$$ x^{'} = F^{'}X + U $$
$$ P^{'} = F^{'}PF^{T} + Q $$ 

### Update
 
Next we take in sensor readings, for a *a postiori* update. The update uses the following equations, which I will explain below: 

$$ y = z- Hx^{'} $$
$$ S = HP^{'}H^{T} + R $$
$$ K = P^{'}H^{T}S^{-1} $$
$$ x = x^{'} + Ky $$
$$ P = (I - KH)P^{'}$$ 

The sensor data is stored as matrices Z and R. 

- Z, sensor matrix (radar, lidar)
- R, covariance matrix of Z  
    - The lidar holds two measurements, the `px` and `py` positions -(2x2) matrix.
    - The radar holds three measurments for `rho, phi, drho` - (3x3) matrix. 

#### Difference Term 
We then calculate the difference term, which can be thought of as `measurement - state`. 

$$ y = z- Hx^{'}$$

#### Innovation Matrix

Matrix S, the innovation covariance, somehow combines P (state covariance), R (measurement covariance) and H. The important part is that S is factored in to compute the optimal Kalman gain K. 

The matrix H is the extraction matrix, and tells us what sensor readings we’d get if x were true and our sensors were perfect. It’s the matrix we use to extract the measurement from the data. If we multiply H times a perfectly correct x, we get a perfectly correct z.

$$ S = HP^{'}H^{T} + R $$

#### Kalman Gain

Next is the Kalman gain, which is derived from the uncertainty matrix P, from above. The Kalman gain can be thought of as a term which weighs our updated estimate towards whatever we have more trust in, our estimate or measurements. 

The bigger is the uncertainty, the more we use our a priori knowledge F about the world in the state mode. The smaller the uncertainty, the more we use the actual measurements. This means that if the measurements are getting noisier than the model will start to use them less; that’s how we are getting a ‘smoothing effect’.

$$ K = P^{'}H^{T}S^{-1} $$

#### Two Types of Measurements 

Normally from here, we could continue the updates for x and P, our matrices of interest, with the following update: 

$$ x = x^{'} + Ky $$
$$ P = (I - KH)P^{'}$$

But there is an issue. When tracking the position and veloctiy with radar/lidar, we get both cartesian and polar coordiante respectively. A direct conversion of polar coordinate to cartesian coordinate gives nonlinearity, since we use $sin$ and $cos$, and so this Kalman filter is actually no longer useful.

## Extended Kalman Filter 

Now we have to linearly map from the polar to the cartesian coordiates, actually now making this an Extended Kalman Filter. The EKF is capable of dealing with multiple sensor types and dimensions. 

The Extended Kalman Filter is almost exactly the same as the Kalman Filter, except for `H`  (technically `F` too, if we stop assuming acceleration is zero, but that is for the next project). 

The reason we cannot use the normal extraction matrix, `H` is because the gaussian noise present in the state estimation will no longer be gaussian for the non-linear polar data. Like so many things in life, the Kalman Filter needs gassians. 

<img src="img/nonlin_gauss.png" width="400" border="10" />

What we need to be able to do is take any sensor measurement, cartesian or polar, and output a consistent state estimate.  

As you are feeding a differently shaped vector (`phi, rho, drho` instead of `x,y`) you need to adjust the `H` matrix. `H` matrix converts your measurement `z` vector that your `y` error term has the same shape as the `x` state. And this is tricky, because your `z` vector comes in different shapes! (1x2 for lidar, 1x3 for radar).

- For Lidar, use a simple 2x4 matrix
- For Radar, use a Jacobian matrix derived from the radar measurement

For radar, we use a first-order Taylor Expansion to obtain a linear approximation of the polar coordiante measurments. In this process, a Jacobian Matrix is produced, which is a linear mapping from the polar to cartesian coordinate system, so that we can estiate `H`. 

We convert our normal $H$ matrix into a $H_j$ matrix, and plug everything back in as before. And since that is the only change, that's it! That is the Extended Kalman Filter. 

### Recap 

- We can use the Kalman Filter to probablistically smooth otherwise noisy data 
- The property of Gaussians makes it such that combining two noisy measurements leads to a more certain estiamte
- The Kalman Filter is a recursive algorithm of predicting and measuring while data is coming in 
- The Extended Kalman Filter allows us to make estimates for non-linear data 
    - takes advantage of linear-non-linear mapping with Jacobian matrices 
    - only need to change one matrix in the calculations - $H$ --> $H_j$ 
This is the linear algebra formular for the EKF: 

#### Prediction - a priori  

$$ x^{'} = F^{'}X + U $$
$$ P^{'} = F^{'}PF^{T} + Q $$ 

#### Update - a postiori
 
$$ y = z- Hx^{'} $$
$$ S = HP^{'}H^{T} + R $$
$$ K = P^{'}H^{T}S^{-1} $$
$$ x = x^{'} + Ky $$
$$ P = (I - KH)P^{'}$$ 

### Sources 

Udacity Self Driving Car Course - Term 2

Past student's blog posts on Medium were a huge help [1](https://medium.com/udacity/udacity-self-driving-car-nanodegree-project-6-extended-kalman-filter-c3eac16c283d) [2](https://medium.com/@kastsiukavets.alena/kalman-filter-extended-kalman-filter-unscented-kalman-filter-dbbd929f83c5) [3](https://medium.com/@mithi/sensor-fusion-and-object-tracking-using-an-extended-kalman-filter-algorithm-part-2-cd20801fbeff) [4](https://medium.com/@mithi/object-tracking-and-fusing-sensor-measurements-using-the-extended-kalman-filter-algorithm-part-1-f2158ef1e4f0) [5](https://medium.com/@tjosh.owoyemi/kalman-filter-predict-measure-update-repeat-20a5e618be66)

[This open source Github book goes deeper into the theory and intuition](https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python)